# Trabalho Final

## Imports

In [87]:
import itertools
import pandas as pd
import numpy as np
import IPython
import random

pd.set_option('display.max_colwidth', None)

## Functions

In [14]:
# Respetivo tempo de cada Tarefa
Inicio = {'T1':38, 'T2':33, 'T3':36, 'T4':20, 'T5':32, 'T6':29, 'T7':46, 'T8':38, 'T9':34, 'T10':40}

# Coluna (Respetiva máquina e qual ordem da tarefa)
colunas = ["M1_T1", "M1_T2", "M1_T3", "M1_T4", "M2_T1", "M2_T2", "M2_T3", "M3_T1", "M3_T2", "M3_T3"]

In [3]:
def DemoraMaquina(df_permutations):
    df_permutations["Maq1"] = df_permutations[["M1_T1", "M1_T2", "M1_T3", "M1_T4"]].apply(lambda row: sum(Inicio.get(valor) for valor in row), axis=1)
    df_permutations["Maq2"] = df_permutations[["M2_T1", "M2_T2", "M2_T3"]].apply(lambda row: sum(Inicio.get(valor) for valor in row), axis=1)
    df_permutations["Maq3"] = df_permutations[["M3_T1", "M3_T2", "M3_T3"]].apply(lambda row: sum(Inicio.get(valor) for valor in row), axis=1)
    df_permutations["MaquinaDemora"] = df_permutations[["Maq1", "Maq2", "Maq3"]].max(axis=1)
    df_permutations.drop(["Maq1", "Maq2", "Maq3"], axis=1, inplace=True)

In [4]:
def RespeitaRestricao(df_permutations):    
    df_permutations["Soma_M1"] = 0
    df_permutations["Soma_M2"] = 0
    df_permutations["Soma_M3"] = 0

    for index, row in df_permutations.iterrows():
        soma_m1 = 0
        m1_critico = 0
        m1_iterado = 0
        soma_m2 = 0
        m2_critico = 0
        m2_iterado = 0
        soma_m3 = 0
        m3_critico = 0
        m3_iterado = 0

        for i in range(1, 5):
            tarefa_m1 = row[f"M1_T{i}"]
            if tarefa_m1 in ["T3", "T4", "T5"]:
                m1_critico += 1
                soma_m1 += m1_iterado
                soma_m1 += Inicio.get(tarefa_m1)
                m1_iterado = 0
            else:
                m1_iterado += Inicio.get(tarefa_m1, 0)
        if m1_critico == 0:
            soma_m1 = 0

        for i in range(1, 4):
            tarefa_m2 = row[f"M2_T{i}"]
            if tarefa_m2 in ["T3", "T4", "T5"]:
                m2_critico += 1
                soma_m2 *= m2_iterado
                soma_m2 += Inicio.get(tarefa_m2)
                m2_iterado = 0
            else:
                m2_iterado += Inicio.get(tarefa_m1, 0)
        if m2_critico == 0:
            soma_m2 = 0

        for i in range(1, 4):
            tarefa_m3 = row[f"M3_T{i}"]
            if tarefa_m3 in ["T3", "T4", "T5"]:
                m3_critico += 1
                soma_m3 += m3_iterado
                soma_m3 += Inicio.get(tarefa_m3)
                m3_iterado = 0
            else:
                m3_iterado += Inicio.get(tarefa_m1, 0)
        if m3_critico == 0:
            soma_m3 = 0

        df_permutations.at[index, "Soma_M1"] = soma_m1
        df_permutations.at[index, "Soma_M2"] = soma_m2
        df_permutations.at[index, "Soma_M3"] = soma_m3
    df_permutations["Maior_Soma"] = df_permutations[["Soma_M1", "Soma_M2", "Soma_M3"]].max(axis=1)
    df_permutations["Restrição Respeitada?"] = df_permutations["Maior_Soma"] <= 96

    df_permutations.drop(["Soma_M1", "Soma_M2", "Soma_M3", "Maior_Soma"], axis=1, inplace=True)

## Pergunta a)

**Neste pergunta será referida uma heurística possível para se aplicar neste problema**

1. Iremos atribuir as 3 tarefas críticas (T3, T4, T5), a cada uma das máquinas, a fim de se conseguir realizar o prazo o mais rápido possivel
2. De seguida, será colocado a tarefa mais rápida na máquina com mais tarefas
3. Por fim, serão atribuidos aleatoriamente as restantes tarefas

## Pergunta b)

In [19]:
Heuristica = ["T3", "T6", "T2", "T9", "T4", "T7", "T1", "T5", "T8", "T10"]

In [20]:
df_heuristica = pd.DataFrame([Heuristica], columns=colunas)
df_heuristica

,M1_T1,M1_T2,M1_T3,M1_T4,M2_T1,M2_T2,M2_T3,M3_T1,M3_T2,M3_T3
0,T3,T6,T2,T9,T4,T7,T1,T5,T8,T10


In [21]:
DemoraMaquina(df_heuristica)
RespeitaRestricao(df_heuristica)
df_heuristica

,M1_T1,M1_T2,M1_T3,M1_T4,M2_T1,M2_T2,M2_T3,M3_T1,M3_T2,M3_T3,MaquinaDemora,Restrição Respeitada?
0,T3,T6,T2,T9,T4,T7,T1,T5,T8,T10,132,True


In [46]:
print("=================================================================")
for coluna in df_permutations.columns[:4]:
    primeira_linha = df_permutations[coluna].iloc[0]
    print(f" ⚫ A \033[1mmáquina 1\033[0m possui a \033[1mtarefa{primeira_linha}\033[0m, que possui uma \033[1mduração de {Inicio.get(primeira_linha)}\033[0m")
for coluna in df_permutations.columns[4:7]:
    primeira_linha = df_permutations[coluna].iloc[0]
    print(f" ⚫ A \033[1mmáquina 2\033[0m possui a \033[1mtarefa{primeira_linha}\033[0m, que possui uma \033[1mduração de {Inicio.get(primeira_linha)}\033[0m")
for coluna in df_permutations.columns[7:]:
    primeira_linha = df_permutations[coluna].iloc[0]
    print(f" ⚫ A \033[1mmáquina 3\033[0m possui a \033[1mtarefa{primeira_linha}\033[0m, que possui uma \033[1mduração de {Inicio.get(primeira_linha)}\033[0m")
print("=================================================================")

 ⚫ A máquina 1 possui a tarefaT3, que possui uma duração de 36
 ⚫ A máquina 1 possui a tarefaT6, que possui uma duração de 29
 ⚫ A máquina 1 possui a tarefaT2, que possui uma duração de 33
 ⚫ A máquina 1 possui a tarefaT9, que possui uma duração de 34
 ⚫ A máquina 2 possui a tarefaT4, que possui uma duração de 20
 ⚫ A máquina 2 possui a tarefaT7, que possui uma duração de 46
 ⚫ A máquina 2 possui a tarefaT1, que possui uma duração de 38
 ⚫ A máquina 3 possui a tarefaT5, que possui uma duração de 32
 ⚫ A máquina 3 possui a tarefaT8, que possui uma duração de 38
 ⚫ A máquina 3 possui a tarefaT10, que possui uma duração de 40


É possível visualizar que a restrição foi respeitada

## Pergunta c)

In [43]:
Cromossoma = '[' + ', '.join([f"'{valor}'" for valor in Heuristica]) + ']'
print(Cromossoma)

['T3', 'T6', 'T2', 'T9', 'T4', 'T7', 'T1', 'T5', 'T8', 'T10']


Nota:
- As 4 primeiras tarefas correspondem à máquina 1
- Da 5 à 7 tarefas correspondem à máquina 2
- As restantes tarefas correspondem à máquina 3

## Pergunta d)

**Experiência com vários cromossomas**

### Crossover Uniforme

In [145]:
import random
import copy

heuristica = ["T3", "T6", "T2", "T9", "T4", "T7", "T1", "T5", "T8", "T10"]

pai1 = copy.deepcopy(heuristica)
pai2 = copy.deepcopy(heuristica)
random.shuffle(pai2)

filho1 = []
filho2 = []

for i in range(len(heuristica)):
    if random.random() < 0.5:  # Seleciona aleatoriamente entre os pais
        filho1.append(pai1[i])
        filho2.append(pai2[i])
    else:
        filho1.append(pai2[i])
        filho2.append(pai1[i])

print("Filho 1:", filho1)
print("Filho 2:", filho2)

Filho 1: ['T3', 'T5', 'T2', 'T4', 'T4', 'T2', 'T10', 'T5', 'T8', 'T9']
Filho 2: ['T1', 'T6', 'T7', 'T9', 'T8', 'T7', 'T1', 'T6', 'T3', 'T10']


In [146]:
df_filhos = pd.DataFrame()

for i, coluna in enumerate(df_permutations.columns):
    df_filhos[coluna] = [filho1[i], filho2[i]]
DemoraMaquina(df_filhos)
RespeitaRestricao(df_filhos)
df_filhos

,M1_T1,M1_T2,M1_T3,M1_T4,M2_T1,M2_T2,M2_T3,M3_T1,M3_T2,M3_T3,MaquinaDemora,Restrição Respeitada?
0,T3,T5,T2,T4,T4,T2,T10,T5,T8,T9,121,False
1,T1,T6,T7,T9,T8,T7,T1,T6,T3,T10,147,True


### Crossover a 1 Ponto

In [124]:
Heuristica = ["T3", "T6", "T2", "T9", "T4", "T7", "T1", "T5", "T8", "T10"]

In [143]:
def crossover_um_ponto(pai1, pai2):
    ponto_corte = random.randint(1, len(pai1) - 1)
    filho1 = pai1[:ponto_corte] + pai2[ponto_corte:]
    filho2 = pai2[:ponto_corte] + pai1[ponto_corte:]
    print(f"K={ponto_corte}")
    return filho1, filho2

pai1 = copy.deepcopy(heuristica)
pai2 = copy.deepcopy(heuristica)
random.shuffle(pai2)

filho1, filho2 = crossover_um_ponto(pai1, pai2)
print("Filho 1:", filho1)
print("Filho 2:", filho2)

K=2
Filho 1: ['T3', 'T6', 'T10', 'T2', 'T3', 'T7', 'T8', 'T4', 'T6', 'T5']
Filho 2: ['T1', 'T9', 'T2', 'T9', 'T4', 'T7', 'T1', 'T5', 'T8', 'T10']


In [144]:
df_filhos = pd.DataFrame()

for i, coluna in enumerate(df_permutations.columns):
    df_filhos[coluna] = [filho1[i], filho2[i]]
DemoraMaquina(df_filhos)
RespeitaRestricao(df_filhos)
df_filhos

,M1_T1,M1_T2,M1_T3,M1_T4,M2_T1,M2_T2,M2_T3,M3_T1,M3_T2,M3_T3,MaquinaDemora,Restrição Respeitada?
0,T3,T6,T10,T2,T3,T7,T8,T4,T6,T5,138,True
1,T1,T9,T2,T9,T4,T7,T1,T5,T8,T10,139,True


## Programação Auxiliar

**Aqui encontra-se apenas código usado para teste**

In [ ]:
Coluna = ["M1_T1", "M1_T2", "M1_T3", "M1_T4", "M2_T1", "M2_T2", "M2_T3", "M3_T1", "M3_T2", "M3_T3"]

In [ ]:
Trabalhos = []
for i in range(10):
    Trabalhos.append(f"T{i+1}")
Trabalhos

In [ ]:
Inicio = {'T1':38, 'T2':33, 'T3':36, 'T4':20, 'T5':32, 'T6':29, 'T7':46, 'T8':38, 'T9':34, 'T10':40}
permutation_list = list(itertools.permutations(Trabalhos))

In [ ]:
x1 = 38+29+36+20
x2 = 38+33+46
x3 = 32+34+40

max(x1, x2, x3)

In [ ]:
Inicio.get("T1")

In [ ]:
df_permutations = pd.DataFrame(permutation_list, columns=Coluna)

In [ ]:
df_permutations

In [ ]:
df_permutations["Maq1"] = df_permutations[["M1_T1", "M1_T2", "M1_T3", "M1_T4"]].apply(lambda row: sum(Inicio.get(valor) for valor in row), axis=1)
df_permutations["Maq2"] = df_permutations[["M2_T1", "M2_T2", "M2_T3"]].apply(lambda row: sum(Inicio.get(valor) for valor in row), axis=1)
df_permutations["Maq3"] = df_permutations[["M3_T1", "M3_T2", "M3_T3"]].apply(lambda row: sum(Inicio.get(valor) for valor in row), axis=1)

In [ ]:
df_permutations["MaquinaDemora"] = df_permutations[["Maq1", "Maq2", "Maq3"]].max(axis=1)
df_permutations.drop(["Maq1", "Maq2", "Maq3"], axis=1, inplace=True)

In [ ]:
df_permutations.head(10)

In [ ]:
df_permutations = df_permutations.head(10)

In [ ]:
RespeitaRestricao(df_permutations)

In [ ]:
df_permutations

- - -